WEB SCRAPING PARA RECOLECTAR IMAGENES

In [ ]:
from googlesearch import search # libreria para hacer busquedas en google
import os # libreria para interactuar con el sistema operativo
import requests # libreria para hacer peticiones http
from bs4 import BeautifulSoup # libreria para hacer web scraping
from tqdm import tqdm # libreria para mostrar barras de progreso
from sklearn.model_selection import train_test_split # libreria para dividir los datos en entrenamiento y prueba
from concurrent.futures import ThreadPoolExecutor # libreria para hacer peticiones http de manera concurrente
from PIL import Image # libreria para trabajar con imagenes
from io import BytesIO # libreria para trabajar con bytes 
import random # libreria para generar numeros aleatorios
import shutil # libreria para trabajar con archivos y directorios
import time # libreria para trabajar con el tiempo

In [ ]:
# Define los nombres de las clases
clases = ["cactus", "cilantro", "eucalipto", "girasol", "hierbabuena", "manzanilla", "rosa"]

In [3]:
# Directorio base del dataset
DATASET_DIR = r"C:\NC\Universidad\6° Semestre\SIS421 (INTELIGENCIA ARTIFICIAL II)\Notebooks\medicinal_plants_project\notebooks\WebScraping\datasetImagenes"
TRAIN_DIR = os.path.join(DATASET_DIR, "train")
VAL_DIR = os.path.join(DATASET_DIR, "val")

IMAGENES_POR_CLASE = 50  # Número máximo de imágenes por clase

In [4]:
# Crear estructura de directorios
def crear_estructura_directorios(base_dir, clases):
    for split in ["train", "val"]:
        split_dir = os.path.join(base_dir, split)
        os.makedirs(split_dir, exist_ok=True)
        for clase in clases:
            os.makedirs(os.path.join(split_dir, clase), exist_ok=True)

crear_estructura_directorios(DATASET_DIR, clases)

In [ ]:
# Obtener URLs de Google para cada clase
def obtener_urls_google(clase, num_paginas=3):
    """
    Busca URLs de páginas relacionadas con una clase usando Google Search.
    """
    query = f"{clase} planta imagen"  # Consulta de búsqueda
    urls = []

    for url in search(query, num_results=num_paginas * 10, lang="es"): # Buscar en Google y obtener los primeros 10 resultados
        urls.append(url) # Agregar URL a la lista
        
        # Agregar un retraso entre las solicitudes para evitar bloqueo por parte de Google
        time.sleep(3)  # 3 segundos de retraso, puedes ajustar según sea necesario

    return urls

In [ ]:
# Descargar imágenes desde URLs específicas
def descargar_imagen(url, clase, carpeta_destino):
    """
    Descarga y guarda una imagen si cumple con los requisitos.
    """
    try:
        #headers = {"User-Agent": "Mozilla/5.0"}
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36" }
        #web scraping
        response = requests.get(url, headers=headers, timeout=10) # Realizar petición GET
        if response.status_code == 200:
            img = Image.open(BytesIO(response.content)) # Abrir la imagen desde los bytes de la respuesta
            if img.format in ["JPEG", "PNG"] and min(img.size) > 200:  # Filtrar imágenes pequeñas
                ruta_destino = os.path.join(carpeta_destino, f"{clase}_{random.randint(10000, 99999)}.jpg")
                img.save(ruta_destino)
                return ruta_destino
    except Exception as e:
        print(f"Error al descargar {url}: {e}")
    return None

In [ ]:
# Extraer imágenes desde múltiples URLs
def extraer_imagenes(urls, clase, carpeta_destino, max_imagenes):
    """
    Extrae imágenes de una lista de URLs hasta alcanzar el número máximo especificado.
    """
    headers = {"User-Agent": "Mozilla/5.0"}
    rutas_guardadas = []
    with ThreadPoolExecutor(max_workers=10) as executor: # Crear un grupo de subprocesos
        for url in urls:
            try:
                #web scraping
                response = requests.get(url, headers=headers, timeout=10)
                soup = BeautifulSoup(response.content, "html.parser")
                imagenes = soup.find_all("img")
                tareas = [
                    executor.submit(descargar_imagen, img.get("src"), clase, carpeta_destino)
                    for img in imagenes if img.get("src")
                ]
                for tarea in tqdm(tareas, desc=f"Procesando imágenes de {url}"):
                    resultado = tarea.result()
                    if resultado:
                        rutas_guardadas.append(resultado)
                        if len(rutas_guardadas) >= max_imagenes:
                            break
                if len(rutas_guardadas) >= max_imagenes:
                    break
            except Exception as e:
                print(f"Error procesando {url}: {e}")
    return rutas_guardadas

In [ ]:
# Dividir datos en entrenamiento y validación, y organizarlos
def dividir_y_organizar_datos(clases, max_imagenes):
    """
    Descarga imágenes para cada clase, las divide en conjuntos de entrenamiento
    y validación, y las organiza en carpetas correspondientes.
    """
    for clase in clases:
        print(f"Buscando imágenes para la clase: {clase}")
        urls = obtener_urls_google(clase) # Obtener URLs de Google
        carpeta_temp = f"{clase}_temp" 
        os.makedirs(carpeta_temp, exist_ok=True)

        # Descargar imágenes hasta alcanzar el límite
        imagenes_descargadas = extraer_imagenes(urls, clase, carpeta_temp, max_imagenes=max_imagenes)

        # Verificar si se descargaron imágenes
        if not imagenes_descargadas:
            print(f"Advertencia: No se encontraron imágenes para la clase '{clase}'.")
            shutil.rmtree(carpeta_temp)
            continue  # Saltar a la siguiente clase

        # Dividir en train y val
        train_imgs, val_imgs = train_test_split(imagenes_descargadas, test_size=0.2, random_state=42)

        # Mover imágenes a carpetas finales
        train_clase_dir = os.path.join(TRAIN_DIR, clase)
        val_clase_dir = os.path.join(VAL_DIR, clase)
        for img_path in train_imgs:
            os.rename(img_path, os.path.join(train_clase_dir, os.path.basename(img_path)))
        for img_path in val_imgs:
            os.rename(img_path, os.path.join(val_clase_dir, os.path.basename(img_path)))

        # Eliminar carpeta temporal de manera recursiva
        shutil.rmtree(carpeta_temp)
        print(f"Clase '{clase}' completada: {len(train_imgs)} para entrenamiento, {len(val_imgs)} para validación.")

In [9]:
# Ejecutar el proceso
dividir_y_organizar_datos(clases, max_imagenes=IMAGENES_POR_CLASE)

Buscando imágenes para la clase: cactus
Error procesando https://depositphotos.com/es/photos/cactus.html: HTTPSConnectionPool(host='depositphotos.com', port=443): Max retries exceeded with url: /es/photos/cactus.html (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000022030FCC760>, 'Connection to depositphotos.com timed out. (connect timeout=10)'))
Error procesando https://es.123rf.com/imagenes-de-archivo/cactus.html: HTTPSConnectionPool(host='es.123rf.com', port=443): Read timed out. (read timeout=10)
Error al descargar data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVQYV2NgYAAAAAMAAWgmWQ0AAAAASUVORK5CYII=: No connection adapters were found for 'data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVQYV2NgYAAAAAMAAWgmWQ0AAAAASUVORK5CYII='Error al descargar data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVQYV2NgYAAAAAMAAWgmWQ0AAAAASUVORK5CYII=: No connection adapters were found fo

Procesando imágenes de https://www.pngwing.com/es/search?q=cactus:   0%|          | 0/61 [00:00<?, ?it/s]

Error al descargar data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVQYV2NgYAAAAAMAAWgmWQ0AAAAASUVORK5CYII=: No connection adapters were found for 'data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVQYV2NgYAAAAAMAAWgmWQ0AAAAASUVORK5CYII='
Error al descargar data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVQYV2NgYAAAAAMAAWgmWQ0AAAAASUVORK5CYII=: No connection adapters were found for 'data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVQYV2NgYAAAAAMAAWgmWQ0AAAAASUVORK5CYII='
Error al descargar data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVQYV2NgYAAAAAMAAWgmWQ0AAAAASUVORK5CYII=: No connection adapters were found for 'data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVQYV2NgYAAAAAMAAWgmWQ0AAAAASUVORK5CYII='
Error al descargar data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAAEAAAABCAQAAAC1HAwCAAAAC0lEQVQYV2NgYAAAAAMAAWgmWQ0AAAAASUVORK5CY

Procesando imágenes de https://www.pngwing.com/es/search?q=cactus: 100%|██████████| 61/61 [00:00<00:00, 154.99it/s]

Error al descargar https://www.pngwing.com/public/css/pngwing.svg: cannot identify image file <_io.BytesIO object at 0x0000022030AA04A0>


...

Error al descargar https://assets-cdn.123rf.com/ui-components/assets/svg/collection.svg: cannot identify image file <_io.BytesIO object at 0x0000022030AC7330>
Error al descargar https://assets-cdn.123rf.com/ui-components/assets/svg/collection.svg: cannot identify image file <_io.BytesIO object at 0x0000022030DA08B0>
Error al descargar https://assets-cdn.123rf.com/ui-components/assets/svg/collection.svg: cannot identify image file <_io.BytesIO object at 0x0000022031B60090>
Error al descargar https://assets-cdn.123rf.com/ui-components/assets/svg/collection.svg: cannot identify image file <_io.BytesIO object at 0x0000022031B818A0>
Error al descargar https://assets-cdn.123rf.com/ui-components/assets/svg/collection.svg: cannot identify image file <_io.BytesIO object at 0x0000022031B9E930>
Error al descargar https://assets-cdn.123rf.com/ui-components/assets/svg/chevron-down.svg: cannot identify image file <_io.BytesIO object at 0x0000022031B80EF0>


Procesando imágenes de https://es.123rf.com/imagenes-de-archivo/planta_de_rosa.html: 100%|██████████| 292/292 [00:06<00:00, 43.05it/s]

Error al descargar https://assets-cdn.123rf.com/ui-components/assets/svg/chevron-down.svg: cannot identify image file <_io.BytesIO object at 0x0000022030AA0810>
Error al descargar https://assets-cdn.123rf.com/ui-components/assets/svg/chevron-down.svg: cannot identify image file <_io.BytesIO object at 0x0000022030AC6C50>
Error al descargar https://assets-cdn.123rf.com/ui-components/assets/svg/chevron-down.svg: cannot identify image file <_io.BytesIO object at 0x000002203119F0B0>
Error al descargar https://assets-cdn.123rf.com/ui-components/assets/svg/chevron-down.svg: cannot identify image file <_io.BytesIO object at 0x0000022030AA33D0>
Error al descargar https://assets-cdn.123rf.com/ui-components/assets/svg/twitter-v2.svg: cannot identify image file <_io.BytesIO object at 0x000002203119EBB0>
Error al descargar https://assets-cdn.123rf.com/ui-components/assets/svg/facebook.svg: cannot identify image file <_io.BytesIO object at 0x0000022030AA2840>
Error al descargar https://assets-cdn.12


Procesando imágenes de https://es.dreamstime.com/foto-de-archivo-planta-de-la-flor-del-rosa-salvaje-image47965576: 100%|██████████| 32/32 [00:01<00:00, 21.98it/s]
Procesando imágenes de https://www.ecologiaverde.com/flores-rosas-4138.html: 100%|██████████| 26/26 [00:01<00:00, 14.92it/s]
Procesando imágenes de https://www.pexels.com/es-es/buscar/rosas%20rosadas/: 0it [00:00, ?it/s]
Procesando imágenes de https://es.pngtree.com/so/rosa: 0it [00:00, ?it/s]
Procesando imágenes de https://pxhere.com/es/photo/1606296:  56%|█████▌    | 10/18 [00:01<00:00,  9.97it/s]


Clase 'rosa' completada: 40 para entrenamiento, 10 para validación.


In [ ]:
# Mostrar la estructura de directorios resultante